# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import PIL
import cv2
import datetime
import os
import matplotlib.pyplot as plt

In [2]:
!pip install keras

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
#train_doc = np.random.permutation(open('/datasets/Project_data/train.csv').readlines())
#val_doc = np.random.permutation(open('/datasets/Project_data/val.csv').readlines())
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

batch_size = 64 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def send_batch_data(source_path, folder_list, batch_size, img_idx, x, y, z, t, batch, batch_data, batch_labels):
    for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes
            #image = cv2.resize(image, (y, z))
            if image.shape[1] == 160:
                image = cv2.resize(image[:,20:140,:],(y,z)).astype(np.float32)
            else:
                image = cv2.resize(image,(y,z)).astype(np.float32)

            batch_data[folder,idx,:,:,0] = cv2.normalize(image[:,:,0], None, alpha=100, beta=255, 
                                                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            batch_data[folder,idx,:,:,1] = cv2.normalize(image[:,:,1], None, alpha=100, beta=255, 
                                                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            batch_data[folder,idx,:,:,2] = cv2.normalize(image[:,:,2], None, alpha=100, beta=255, 
                                                                 norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            
        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    return batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    #create a list of image numbers you want to use for a particular video
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29] 

    x = len(img_idx) # number of images in the seq
    y = 84 # image_height
    z = 84 # image_width
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            # Calling the function which forms the batch data
            batch_data, batch_labels = send_batch_data(source_path, folder_list, batch_size, img_idx, x, y, z, t, 
                                                       batch, batch_data, batch_labels)
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        rem_img_size = len(folder_list) - batch_size * num_batches
        batch_num = num_batches
        batch_data = np.zeros((rem_img_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((rem_img_size,5)) # batch_labels is the one hot representation of the output
        
        # Calling the function which forms the batch data
        batch_data, batch_labels = send_batch_data(source_path, folder_list, rem_img_size, img_idx, x, y, z, t, 
                                                   batch_num, batch_data, batch_labels)
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [11]:
curr_dt_time = datetime.datetime.now()
train_path = '/datasets/Project_data//train'
#val_path = '/datasets/Project_data//val'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


In [19]:
path_to_training_dataset=val_path
import Augmentor
for i in (0,num_train_sequences):
    print((str(val_path) + "/" + val_doc[i].strip().split(';')[0]) + "/")
    p = Augmentor.Pipeline(str(val_path) + "/" + val_doc[i].strip().split(';')[0] + "/")
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    p.sample(30) ## We are adding 30 samples per class to make sure that none of the classes are sparse.

Executing Pipeline:   0%|          | 0/30 [00:00<?, ? Samples/s]

Project_data/val/WIN_20180926_17_24_20_Pro_Thumbs_Down_new/
Initialised with 30 image(s) found.
Output directory set to Project_data/val/WIN_20180926_17_24_20_Pro_Thumbs_Down_new/output.

AttributeError: 'FileNotFoundError' object has no attribute 'message'

In [8]:
# plot a sample image
#import glob
#import os
#from skimage import io

#img_path = os.path.join(train_path, train_doc[2].strip().split(';')[0], '*')
#print(img_path)
#img_path = glob.glob(img_path)
#print(img_path)

# access some element (a file) from the list
#for i in range(0,29):
#    image = io.imread(img_path[i])
#    plt.imshow(image)
    #ax.imshow(image)
#    plt.show()

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
img_height = 84 
img_width = 84
num_img = 18

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model = Sequential()

model.add(Conv3D(32, kernel_size=(3, 3, 3),activation='relu',padding='same', input_shape=(num_img,img_height,img_width,3)))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Conv3D(128, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Conv3D(256, kernel_size=(3, 3, 3),activation='relu', padding='same'))
model.add(BatchNormalization()),
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 84, 84, 32)    128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 42, 42, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 42, 42, 64)     55360     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 42, 42, 64)     256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 21, 21, 64)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 4, 21, 21, 128)    2

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [11]:
#optimiser = tf.keras.optimizers.Adam() #write your optimizer
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 84, 84, 32)    128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 42, 42, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 42, 42, 64)     55360     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 42, 42, 64)     256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 21, 21, 64)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 4, 21, 21, 128)    2

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0,min_lr=0.0001)

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(validation_steps) 
print(num_val_sequences)
print(batch_size)
# running only one batch 
#validation_steps = steps_per_epoch = 1

2
100
64


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  /datasets/Project_data//train ; batch size = 64
Epoch 1/50
11/11 [==============================] - ETA: 0s - loss: 4.9963 - categorical_accuracy: 0.2036Source path =  /datasets/Project_data//val ; batch size = 64

Epoch 00001: val_loss improved from inf to 22.24662, saving model to model_init_2021-07-2710_50_42.694723/model-00001-4.99628-0.20362-22.24662-0.20000.h5
11/11 [==============================] - 26s 2s/step - loss: 4.9963 - categorical_accuracy: 0.2036 - val_loss: 22.2466 - val_categorical_accuracy: 0.2000
Epoch 2/50
11/11 [==============================] - ETA: 0s - loss: 2.3726 - categorical_accuracy: 0.3499
Epoch 00002: val_loss improved from 22.24662 to 15.85768, saving model to model_init_2021-07-2710_50_42.694723/model-00002-2.37263-0.34992-15.85768-0.21000.h5
11/11 [==============================] - 27s 2s/step - loss: 2.3726 - categorical_accuracy: 0.3499 - val_loss: 15.8577 - 